In [54]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re

In [55]:
# Ruta del archivo .pkl
file_path = r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\Proyecto Final\datasets\Yelp\business.pkl'

# Cargar el archivo .pkl
df_business_yelp_base = pd.read_pickle(file_path)

In [73]:
df_business_y = df_business_yelp_base

In [74]:
len(df_business_y)

150346

### Eliminamos las columnas que tienen un 98% de valores nulos (son columnas erroneas)

In [75]:
# Eliminar columnas duplicadas
df_business_y = df_business_y.loc[:, ~df_business_y.columns.duplicated()]

# Eliminar las columnas 'state', 'postal_code', y 'hours'
df_business_y.drop(columns=['state', 'postal_code'], inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_15784\1179359240.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_business_y.drop(columns=['state', 'postal_code'], inplace=True)


### Rellenamos las filas que tienen valores nulos en "State" harcodeandolo ya que son solo 3

In [76]:
# Diccionario con indice y valores para rellenar esos vacios
valores_para_reemplazo = {
    0: "California",  
    1: "Affton",    
    2: "Arizona", 
}

# Rellenar valores nulos en 'state' con estado específico
for index, value in valores_para_reemplazo.items():
    df_business_y.loc[index, 'state'] = value

C:\Users\Usuario\AppData\Local\Temp\ipykernel_15784\3362674017.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_business_y.loc[index, 'state'] = value


### Filtramos para obtener solo registros de la ciudades de California


In [77]:
# Lista de ciudades de California
california_cities = [
    'Los Angeles', 'San Francisco', 'San Diego', 'San Jose', 'Sacramento', 'Fresno', 'Long Beach', 'Oakland',
    'Bakersfield', 'Anaheim', 'Santa Ana', 'Riverside', 'Stockton', 'Chula Vista', 'Irvine', 'Fremont', 
    'San Bernardino', 'Modesto', 'Fontana', 'Moreno Valley', 'Huntington Beach', 'Glendale', 'Oxnard', 
    'Ontario', 'Rancho Cucamonga', 'Santa Clarita', 'Oceanside', 'Garden Grove', 'Elk Grove', 'Corona',
    'Lancaster', 'Palmdale', 'Hayward', 'Salinas', 'Pomona', 'Sunnyvale', 'Escondido', 'Torrance', 
    'Pasadena', 'Orange', 'Fullerton', 'Thousand Oaks', 'Visalia', 'Simi Valley', 'Concord', 
    'Roseville', 'Victorville', 'Vallejo', 'Berkeley', 'Fairfield'
]

# Convertir todas las ciudades a minúsculas para estandarizar
california_cities_lower = [city.lower() for city in california_cities]

# Limpiar la columna 'city' en df_business_y
df_business_y['city'] = df_business_y['city'].astype(str).str.lower().str.strip()

# Verificar algunas ciudades únicas en df_business_y
print("Algunas ciudades únicas en df_business_y:")
print(df_business_y['city'].unique()[:20])

# Filtrar el DataFrame para mantener solo las filas donde la ciudad esté en la lista de California
df_business_y = df_business_y[df_business_y['city'].isin(california_cities_lower)]

# Verificar el número de registros después del filtrado
print(f"\nNúmero de registros en df_business_y: {df_business_y.shape[0]}")


Algunas ciudades únicas en df_business_y:
['santa barbara' 'affton' 'tucson' 'philadelphia' 'green lane'
 'ashland city' 'brentwood' 'st. petersburg' 'nashville' "land o' lakes"
 'tampa bay' 'indianapolis' 'clearwater' 'largo' 'new orleans' 'kenner'
 'edmonton' 'reno' 'newtown' 'white house']

Número de registros en df_business_y: 41


C:\Users\Usuario\AppData\Local\Temp\ipykernel_15784\2128959044.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_business_y['city'] = df_business_y['city'].astype(str).str.lower().str.strip()


### Desanidamos la columna attributes

In [78]:
# Desanidar la columna 'attributes'
df_attributes_desanidada = pd.json_normalize(df_business_y['attributes'])

# Combinar las nuevas columnas desanidadas con el DataFrame original
df_business_y = pd.concat([df_business_y, df_attributes_desanidada], axis=1)

# Eliminar la columna original 'attributes' si ya no es necesaria
df_business_y.drop(columns=['attributes'], inplace=True)

In [79]:
# Definir el patrón de expresión regular para identificar categorías relacionadas con la gastronomía
palabra = r'\b(restaurant|cafe|delivery|diner|bistro|takeout|bar|pub|grill|pizzeria|coffee|bakery|food|eatery|sandwich|snack)\b'

# Filtrar las categorías que coinciden con el patrón de gastronomía, manejando NaN
df_business_yelp = df_business_y[df_business_y['categories'].str.contains(palabra, case=False, regex=True, na=False)]

C:\Users\Usuario\AppData\Local\Temp\ipykernel_15784\194133525.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_business_yelp = df_business_y[df_business_y['categories'].str.contains(palabra, case=False, regex=True, na=False)]


In [80]:
len(df_business_yelp)

12

codigo para ver columnas desanidadas de attributes

In [65]:
# Nombre de la columna a partir de la cual deseas ver los nombres de las columnas
columna_inicio = 'state'  # Reemplaza con el nombre real de tu columna

# Verificar si la columna existe en el DataFrame
if columna_inicio in df_business_y.columns:
    # Seleccionar las columnas a partir de 'columna_inicio' usando loc
    columnas_seleccionadas = df_business_y.loc[:, columna_inicio:].columns
    
    # Convertir los nombres de las columnas a una lista
    lista_columnas = columnas_seleccionadas.tolist()
    
    # Mostrar la lista de nombres de las columnas seleccionadas
    print("Nombres de las columnas seleccionadas:", lista_columnas)
else:
    print(f"La columna '{columna_inicio}' no existe en el DataFrame.")

Nombres de las columnas seleccionadas: ['state', 'ByAppointmentOnly', 'BusinessAcceptsCreditCards', 'BikeParking', 'RestaurantsPriceRange2', 'CoatCheck', 'RestaurantsTakeOut', 'RestaurantsDelivery', 'Caters', 'WiFi', 'BusinessParking', 'WheelchairAccessible', 'HappyHour', 'OutdoorSeating', 'HasTV', 'RestaurantsReservations', 'DogsAllowed', 'Alcohol', 'GoodForKids', 'RestaurantsAttire', 'Ambience', 'RestaurantsTableService', 'RestaurantsGoodForGroups', 'DriveThru', 'NoiseLevel', 'GoodForMeal', 'BusinessAcceptsBitcoin', 'Smoking', 'Music', 'GoodForDancing', 'AcceptsInsurance', 'BestNights', 'BYOB', 'Corkage', 'BYOBCorkage', 'HairSpecializesIn', 'Open24Hours', 'RestaurantsCounterService', 'AgesAllowed', 'DietaryRestrictions']


### Guardar dataframe en parquet

In [66]:
#df_business_y.to_parquet(r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\repo\epicurean_project\DATA\data parcial\locales_gastronomicos_yelp.parquet')

codigo que desanida la col attributes solo con las columnas que queremos

In [67]:
"""
# 1. Definir las claves que deseas extraer
keys_to_extract = ['key1', 'key2', 'key3']  # Reemplaza con tus claves

# 2. Desanidar solo las claves seleccionadas
df_attributes_desanidada = pd.json_normalize(df_business_y['attributes'])[keys_to_extract]

# 3. Combinar las nuevas columnas con el DataFrame original
df_business_y = pd.concat([df_business_y, df_attributes_desanidada], axis=1)

# 4. Eliminar la columna original 'attributes' si ya no es necesaria
df_business_y.drop(columns=['attributes'], inplace=True)
"""

"\n# 1. Definir las claves que deseas extraer\nkeys_to_extract = ['key1', 'key2', 'key3']  # Reemplaza con tus claves\n\n# 2. Desanidar solo las claves seleccionadas\ndf_attributes_desanidada = pd.json_normalize(df_business_y['attributes'])[keys_to_extract]\n\n# 3. Combinar las nuevas columnas con el DataFrame original\ndf_business_y = pd.concat([df_business_y, df_attributes_desanidada], axis=1)\n\n# 4. Eliminar la columna original 'attributes' si ya no es necesaria\ndf_business_y.drop(columns=['attributes'], inplace=True)\n"